In [4]:
import pandas as pd
from tqdm import tqdm

In [3]:
# 대상 기업 정보 불러오기
fs_all = pd.read_excel('data/fs-account.xlsx', dtype = str)

In [7]:
# 대상 기업 정보 불러오기
corp_list = pd.read_excel('data/etf_fs_target_company.xlsx', dtype = str)

In [89]:
# 결과 데이터
data = pd.DataFrame()

for idx, v in tqdm(corp_list.iterrows()) :
    corp_code, stock_name, stock_code = v

    tmp = fs_all.loc[fs_all['corp_code'] == corp_code, :]
    bs = filter_bs_account(tmp)

    data = pd.concat([data, bs])


2301it [01:14, 30.74it/s]


In [101]:
fs_all.loc[fs_all['account_nm'].str.contains('부채 총계'), ['account_id', 'account_nm']].drop_duplicates()

,account_id,account_nm
29258,ifrs-full_EquityAndLiabilities,자본과 부채 총계
37543,ifrs-full_Liabilities,부채 총계
272335,ifrs-full_EquityAndLiabilities,자본 및 부채 총계


In [103]:
liability = (fs_all['account_id'] == 'ifrs-full_Liabilities')
fs_all.loc[liability, ['corp_code']].drop_duplicates()

,corp_code
31,01170962
168,01139266
306,00938688
522,01107665
642,00261735
...,...
461283,00633835
461451,00317104
461598,00651901
461723,00661847


In [ ]:
equity = (fs_all['account_id'] == 'ifrs-full_Equity')
retained_earning = fs_all['account_id'] == 'ifrs-full_RetainedEarnings'

fs_all.loc[equity, ['corp_code']].drop_duplicates()

,corp_code
16,01170962
152,01139266
314,00938688
509,01107665
632,00261735
...,...
461258,00633835
461436,00317104
461585,00651901
461707,00661847


In [126]:
fs_data = fs_all.copy()

In [134]:
# 계정코드

# 자산
bs = fs_data['sj_div'] == 'BS'

# 1. 자산 사이
# 1) 자산 총계
asset = bs & (
    (fs_data['account_id'] == 'ifrs-full_Assets') |
    fs_data['account_id'].str.contains('-표준계정코드 미사용-') & fs_data['account_nm'].str.contains('자산총계')
    )
if len(fs_data[asset]) > 1 :
    asset = bs & fs_data['account_id'].str.contains('ifrs-full_Assets')

# 2) 유동자산
current_asset = bs & (fs_data['account_id'] == 'ifrs-full_CurrentAssets')

# 3) 비유동자산
non_current_asset = bs & (fs_data['account_id'] == 'ifrs-full_NoncurrentAssets')

# 4) 현금 및 현금성자산
cash = bs & (fs_data['account_id'] == 'ifrs-full_CashAndCashEquivalents')

# 5) 매출채권
rcvb_idx = bs & (
    fs_data['account_id'].str.contains('ifrs-full_TradeAndOtherCurrentReceivables') |
    fs_data['account_id'].str.contains('ifrs-full_TradeReceivables') |
    fs_data['account_id'].str.contains('ifrs-full_CurrentTradeReceivables') |
    fs_data['account_id'].str.contains('dart_ShortTermTradeReceivable') |
    fs_data['account_id'].str.contains('-표준계정코드 미사용-') & fs_data['account_nm'].str.contains('매출채권')
)

# if len(fs_data[rcvb_idx]) > 1 :

#     if 'dart_ShortTermTradeReceivable' in fs_data[rcvb_idx]['account_id'].tolist() :
#         receivable = bs & (fs_data['account_id'] == 'dart_ShortTermTradeReceivable')
#     elif 'ifrs-full_CurrentTradeReceivables' in fs_data[rcvb_idx]['account_id'].tolist() :
#         receivable = bs & (fs_data['account_id'] == 'ifrs-full_CurrentTradeReceivables')
#     elif 'ifrs-full_TradeReceivables' in fs_data[rcvb_idx]['account_id'].tolist() :
#         receivable = bs & (fs_data['account_id'] == 'ifrs-full_TradeReceivables')
#     elif 'ifrs-full_TradeAndOtherCurrentReceivables' in fs_data[rcvb_idx]['account_id'].tolist() :
#         receivable = bs & (fs_data['account_id'] == 'ifrs-full_TradeAndOtherCurrentReceivables')
#     else :
#         idx = fs_data[rcvb_idx]['ord'].astype(int).idxmin()
#         receivable = fs_data.index == idx

# else :
receivable = rcvb_idx

# 6) 재고자산
inventory = bs & (
    fs_data['account_id'].str.contains('ifrs-full_Inventories') |
    fs_data['account_id'].str.contains('-표준계정코드 미사용-') & (fs_data['account_nm'] == '재고자산')
)

if len(fs_data[inventory]) > 1 :
    inventory = bs & fs_data['account_id'].str.contains('ifrs-full_Inventories')


# 2. 자본
# 1) 자기자본
equity = bs & (fs_data['account_id'] == 'ifrs-full_Equity')
# 2) 이익잉여금
retained_earning = bs & (fs_data['account_id'] == 'ifrs-full_RetainedEarnings')

# 3. 부채
# 1) 부채총계
liability = bs & (fs_data['account_id'] == 'ifrs-full_Liabilities')
# 2) 유동부채
current_liability = bs & (fs_data['account_id'] == 'ifrs-full_CurrentLiabilities')
# 3) 비유동부채
non_current_liability = bs & (fs_data['account_id'] == 'ifrs-full_NoncurrentLiabilities')

In [ ]:
corp_asset = set(fs_data.loc[asset, 'corp_code'])
corp_current_asset = set(fs_data.loc[current_asset, 'corp_code'])
corp_non_current_asset = set(fs_data.loc[non_current_asset, 'corp_code'])
corp_cash = set(fs_data.loc[cash, 'corp_code'])
corp_receivable = set(fs_data.loc[receivable, 'corp_code'])
corp_inventory = set(fs_data.loc[inventory, 'corp_code'])
corp_equity = set(fs_data.loc[equity, 'corp_code'])
corp_retained_earning = set(fs_data.loc[retained_earning, 'corp_code'])
corp_liability = set(fs_data.loc[liability, 'corp_code'])
corp_current_liability = set(fs_data.loc[current_liability, 'corp_code'])
corp_non_current_liability = set(fs_data.loc[non_current_liability, 'corp_code'])

In [ ]:
print('corp_asset',  len(corp_asset))
print('corp_current_asset', len(corp_current_asset))
print('corp_non_current_asset', len(corp_non_current_asset))
print('corp_cash', len(corp_cash))
print('corp_receivable', len(corp_receivable))
print('corp_inventory', len(corp_inventory))
print('corp_equity', len(corp_equity))
print('corp_retained_earning', len(corp_retained_earning))
print('corp_liability', len(corp_liability))
print('corp_current_liability', len(corp_current_liability))
print('corp_non_current_liability', len(corp_non_current_liability))

corp_asset 2656
corp_current_asset 2564
corp_non_current_asset 2546
corp_cash 2606
corp_receivable 2506
corp_inventory 2253
corp_equity 2648
corp_retained_earning 2613
corp_liability 2656
corp_current_liability 2560
corp_non_current_liability 2563


In [161]:
a = set(fs_data.loc[asset, 'corp_code'])
b = set(fs_data.loc[current_asset, 'corp_code'])
c = set(fs_data.loc[non_current_asset, 'corp_code'])
d = set(fs_data.loc[cash, 'corp_code'])
e = set(fs_data.loc[receivable, 'corp_code'])
f = set(fs_data.loc[inventory, 'corp_code'])
g = set(fs_data.loc[equity, 'corp_code'])
h = set(fs_data.loc[retained_earning, 'corp_code'])
i = set(fs_data.loc[liability, 'corp_code'])
j = set(fs_data.loc[current_liability, 'corp_code'])
k = set(fs_data.loc[non_current_liability, 'corp_code'])

In [ ]:
final_result = list(set.intersection(a,b,c,d,e,f,g,h,i,j,k))

In [163]:
dart_code_list = pd.read_json('data/dart_code_list.json', dtype = str)
dart_code_list[dart_code_list['고유번호'].isin(final_result)].reset_index(drop=True).to_json('data/etf_fs_target_company.json')
dart_code_list[dart_code_list['고유번호'].isin(final_result)].reset_index(drop=True).to_excel('data/etf_fs_target_company.xlsx')

2207